<a href="https://colab.research.google.com/github/Cibranix/Notebook_ABP/blob/main/M%C3%85CIAGGOJ%C3%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MÅCIAGGOJÖ - Sistema de recomendación de productos IKEA®
Alumnos:
- José Manuel Villar Constenla [ABP_5]
- Gonzalo Álvarez Acevedo [ABP_5]
- María Andrea Ugarte Valencia [ABP_5]
- Cibrán Cores Cabaleiro [ABP_5]
- Agustín Suárez Martínez [ABP_5]

##Introducción

El proyecto MÅCIAGGOJÖ consiste en la elaboración de una plataforma de recomendación de productos de IKEA® donde el usuario introduce bien por una imagen o por una descripción el objeto que busca y se le devuelve un listado de ítems similares recogidos en la web oficial de IKEA®.

Para la fase de desarrollo de la interpretación de descripción, se hará uso de un dataset abierto disponible en Kaggle ([ver](https://www.kaggle.com/datasets/crawlfeeds/ikea-us-products-dataset)) que será procesado para eliminar información innecesaria o redundante.

##Procesado del dataset

Primero de todo cargamos el dataset "en sucio", sin procesar los datos. Podemos observar que hay datos que no son muy relevantes a la hora de crear nuestro sistema de recomendación ya como el dataset esta formado por datos de IKEA en Estados Unidos: la moneda es USD, el lenguaje es Inglés y el seller y brand es IKEA. También vamos a suponer que la condición del producto es siempre nuevo, de primera mano, y siempre están disponibles, es decir, hay stock. Entonces, nos quedaremos con las siguientes columnas:
- `product_title`
- `product_url`
- `product_price`
- `breadcrumbs`
- `raw_product_details`
- `sku`

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive/')

originalData = pd.read_json("/content/drive/MyDrive/ABP/src.json")
originalData

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,product_title,product_url,sku,mpn,currency,product_price,product_condition,availability,seller,seller_url,brand,raw_product_details,breadcrumbs,country,language,average_rating,reviews_count
0,JÄRVFJÄLLET Office chair with armrests,https://www.ikea.com/us/en/p/jaervfjaellet-off...,605.106.40,605.106.40,USD,279.00,NewCondition,InStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Furniture/Chairs/Desk chairs/Desk cha...,US,EN,,
1,"DRÖNA Box - black 9 ¾x13 ¾x9 ¾ """,https://www.ikea.com/us/en/p/droena-box-black-...,904.670.65,904.670.65,USD,2.99,NewCondition,InStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Home Décor/Storage boxes & baskets/Pa...,US,EN,4.8,451
2,"BESTIGA Candlestick, set of 3 - black",https://www.ikea.com/us/en/p/bestiga-candlesti...,605.046.77,605.046.77,USD,24.99,NewCondition,InStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Home Décor/Candle holders & candles/C...,US,EN,5,7
3,"EKET Suspension rail 13 3/4 """,https://www.ikea.com/us/en/p/eket-suspension-r...,003.400.47,003.400.47,USD,5.00,NewCondition,InStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Storage & organization/Shelving units...,US,EN,3.8,16
4,"SEVÄRD Tablecloth - dark blue 57x94 """,https://www.ikea.com/us/en/p/sevaerd-tableclot...,103.640.90,103.640.90,USD,19.99,NewCondition,InStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Cookware & tableware/Table linen/Tabl...,US,EN,4.8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11818,SEKTION High cabinet with shelves/2 doors - wh...,https://www.ikea.com/us/en/p/sektion-high-cabi...,594.210.65,594.210.65,USD,297.00,NewCondition,InStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Kitchen & appliances/Kitchen systems/...,US,EN,4.8,4
11819,"BODARP Drawer front - gray-green 15x15 """,https://www.ikea.com/us/en/p/bodarp-drawer-fro...,204.356.24,204.356.24,USD,25.00,NewCondition,InStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Kitchen & appliances/Kitchen systems/...,US,EN,4.5,2
11820,BESTÅ TV bench with doors and drawers - black-...,https://www.ikea.com/us/en/p/besta-tv-bench-wi...,994.359.37,994.359.37,USD,424.00,NewCondition,OutOfStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Storage & organization/TV & media fur...,US,EN,,
11821,GODMORGON / BRÅVIKEN Sink cabinet with 4 drawe...,https://www.ikea.com/us/en/p/godmorgon-bravike...,792.925.43,792.925.43,USD,899.00,NewCondition,OutOfStock,IKEA,,IKEA,"<span class=""range-revamp-product-details__par...",Products/Bathroom/Bathroom vanities/Vanity cab...,US,EN,4,1


Una vez cargado el dataset, vamos a procesar el campo `raw_product_details` que es un campo con etiquetas xml, para obtener los detalles del producto que están entre varias de esas etiquetas.
Podemos observar a continuación como obtener esa columna:

In [2]:
columnaDetallesRaw = originalData["raw_product_details"]
columnaDetallesRaw

0        <span class="range-revamp-product-details__par...
1        <span class="range-revamp-product-details__par...
2        <span class="range-revamp-product-details__par...
3        <span class="range-revamp-product-details__par...
4        <span class="range-revamp-product-details__par...
                               ...                        
11818    <span class="range-revamp-product-details__par...
11819    <span class="range-revamp-product-details__par...
11820    <span class="range-revamp-product-details__par...
11821    <span class="range-revamp-product-details__par...
11822    <span class="range-revamp-product-details__par...
Name: raw_product_details, Length: 11823, dtype: object

Y también podemos observar como obtener la primera tupla de un dataset:

In [3]:
originalData.iloc[0]

product_title                     JÄRVFJÄLLET Office chair with armrests
product_url            https://www.ikea.com/us/en/p/jaervfjaellet-off...
sku                                                           605.106.40
mpn                                                           605.106.40
currency                                                             USD
product_price                                                      279.0
product_condition                                           NewCondition
availability                                                     InStock
seller                                                              IKEA
seller_url                                                              
brand                                                               IKEA
raw_product_details    <span class="range-revamp-product-details__par...
breadcrumbs            Products/Furniture/Chairs/Desk chairs/Desk cha...
country                                            

Combinando los anteriores métodos podemos obtener el primer campo de la primera tupla, es decir, del primer producto de IKEA a analizar con el siguiente código:

In [4]:
descripProduct = originalData.iloc[0]["raw_product_details"]
descripProduct

'<span class="range-revamp-product-details__paragraph">10-year Warranty. Read about the terms in the warranty brochure.</span>/<span class="range-revamp-product-details__paragraph">The adjustable and lockable tilt function increases stability and control in different sitting positions.</span>/<span class="range-revamp-product-details__paragraph">The mesh material of the backrest lets air through to your back during long sitting periods.</span>/<span class="range-revamp-product-details__paragraph">Your back gets the support it needs because the seat depth can be adjusted and the chair has built-in lumbar support.</span>/<span class="range-revamp-product-details__paragraph">You can adjust the headrest and the lumbar support to suit you and your needs.</span>/<span class="range-revamp-product-details__paragraph">The safety casters have a pressure-sensitive brake mechanism that keeps the chair in place when you stand up, and releases automatically when you sit down.</span>/<span class="ran

Podemos observar que el código esta lleno de etiquetas `span` y a nosotros nos van a interesar los que tengan la clase `range-revamp-product-details__paragraph` y la clase `range-revamp-product-details__header notranslate` ya que tienen la descripción del producto y título corto del producto respectivamente.

**Nota** *: Título corto sería, el título en mayusculas, es decir, en el primer producto sería* **JÄRVFJÄLLET**

Pero para poder procesarlo vamos a necesitar concatenarle antes y después una etiqueta `root`, de la siguiente manera:

In [5]:
descripProductDP = "<root>"+descripProduct+"</root>"
descripProductDP

'<root><span class="range-revamp-product-details__paragraph">10-year Warranty. Read about the terms in the warranty brochure.</span>/<span class="range-revamp-product-details__paragraph">The adjustable and lockable tilt function increases stability and control in different sitting positions.</span>/<span class="range-revamp-product-details__paragraph">The mesh material of the backrest lets air through to your back during long sitting periods.</span>/<span class="range-revamp-product-details__paragraph">Your back gets the support it needs because the seat depth can be adjusted and the chair has built-in lumbar support.</span>/<span class="range-revamp-product-details__paragraph">You can adjust the headrest and the lumbar support to suit you and your needs.</span>/<span class="range-revamp-product-details__paragraph">The safety casters have a pressure-sensitive brake mechanism that keeps the chair in place when you stand up, and releases automatically when you sit down.</span>/<span clas

Luego con el siguiente código vamos recorriendo el string obteniendo los valores de los spans cuya clase es `range-revamp-product-details__paragraph` y de esta forma crear el resumen de los detalles del producto.

In [6]:
from re import findall
import xml.etree.ElementTree as ET
root = ET.fromstring(descripProductDP)

textoResumen = ""

for details in root.findall("span"):
  if details.get("class") == "range-revamp-product-details__paragraph":
    textoResumen = textoResumen + details.text

textoResumen

'10-year Warranty. Read about the terms in the warranty brochure.The adjustable and lockable tilt function increases stability and control in different sitting positions.The mesh material of the backrest lets air through to your back during long sitting periods.Your back gets the support it needs because the seat depth can be adjusted and the chair has built-in lumbar support.You can adjust the headrest and the lumbar support to suit you and your needs.The safety casters have a pressure-sensitive brake mechanism that keeps the chair in place when you stand up, and releases automatically when you sit down.The adjustable armrests of this swivel chair can help take the load off your shoulders and neck to reduce tension, strain and pain.The leather ages beautifully and acquires a nice patina over time.This chair has been tested for office use and meets the requirements for durability and stability set forth in the following standards: EN 1335 and ANSI/BIFMA x5.1.May be used with KOLON floo

Una vez obtenida la descripción del primer producto solo tenemos que aplicar lo anterior para todos los elementos de la columna `raw_product_details` y crear una lista con todas estas descripciones en el formato que nos interesa. Pero antes vamos a obtener el campo `range-revamp-product-details__header notranslate` para obtener los títulos.

In [7]:
from re import findall
import xml.etree.ElementTree as ET

titulos = []
for descripcion in columnaDetallesRaw:
  descripProduct = "<root>"+descripcion+"</root>"
  root = ET.fromstring(descripProduct)
  titulo = ""
  for details in root.findall("span"):
    if details.get("class") == "range-revamp-product-details__header notranslate":
      if details.text != None:
        titulo = titulo + details.text + " "
  titulos.append(titulo.strip())

titulos

['JÄRVFJÄLLET',
 'DRÖNA',
 'BESTIGA',
 'EKET',
 'SEVÄRD',
 'BODBYN SEKTION UTRUSTA',
 'ÄPPLARÖ ÄPPLARÖ DUVHOLMEN FRÖSÖN',
 'DRUVFLÄDER',
 'PAX PAX PAX PAX KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT KOMPLEMENT',
 'BLYSKÄR LOBERGET PYNTEN',
 'GULLIGAST',
 'SINARP SEKTION UTRUSTA UTRUSTA',
 'JÄRSTA SEKTION UTRUSTA MAXIMERA UTRUSTA',
 'BEKANT',
 'KUNGSBACKA SEKTION UTRUSTA',
 'LURÖY',
 'HITTARP HITTARP SEKTION UTRUSTA UTRUSTA UTRUSTA',
 'BESTÅ BESTÅ BESTÅ BESTÅ HEDEVIKEN',
 'KNYCK',
 'LÅNGFJÄLL LÅNGFJÄLL LÅNGFJÄLL',
 'BESTÅ BESTÅ BESTÅ BESTÅ BERGSVIKEN ÖSARP BESTÅ',
 'AXSTAD AXSTAD SEKTION MAXIMERA UTRUSTA UTRUSTA',
 'JUBLA',
 'FJÄLLHAVRE',
 'HAVSTORP HAVSTORP SEKTION MAXIMERA MAXIMERA UTRUSTA UTRUSTA UTRUSTA',
 'GRIMSLÖV GRIMSLÖV SEKTION MAXIMERA UTRUSTA UTRUSTA UTRUSTA',
 'ASKERSUND ASKERSUND SEKTION MAXIMERA MAXIMERA',
 'FRÖJERED SEKTION MAXIMERA',
 'KOMPLEMENT',
 'BODARP BODARP SEKTION MAXIMERA MAXIMERA UTRUS

Sin embargo, hay algunos títulos vacios y otros que tienen muchos titulos ya que pertecen a un pack, por lo que vamos a hacer un filtro y quedarnos con los que tengan solo un campo y realizar la suma para ver con cuantos campos nos quedamos. Y también vamos a crear un array de booleanos indicando con cuales nos quedamos.

In [8]:
c = 0
arrayKeep = []

for a in titulos:
  arrayPalabras = a.split(" ")
  if arrayPalabras[0] != '':
    numero = len(arrayPalabras)
  else:
    numero = 0

  if numero == 1:
    c = c + 1
    arrayKeep.append(True)
  else:
    arrayKeep.append(False)

print(c)


4493


Nos quedamos con casi 4500 productos que son suficientes para realizar nuestro sistema de recomendación. Y creamos un nuevo dataset quedandonos solo con esas tuplas que llamamos `newData`.

In [9]:
newData = originalData[arrayKeep]

Ahora sobre este nuevo dataset vamos a obtener las descripciones de todos los productos que se comentó anteriormente:

In [10]:
descripciones = []

for descripcion in newData["raw_product_details"]:
  descripProduct = "<root>"+descripcion+"</root>"
  root = ET.fromstring(descripProduct)
  textoResumen = ""
  titulo = ""
  for details in root.findall("span"):
    if details.get("class") == "range-revamp-product-details__paragraph":
      textoResumen = textoResumen + details.text + " "

  descripciones.append(textoResumen.strip())

descripciones

['10-year Warranty. Read about the terms in the warranty brochure. The adjustable and lockable tilt function increases stability and control in different sitting positions. The mesh material of the backrest lets air through to your back during long sitting periods. Your back gets the support it needs because the seat depth can be adjusted and the chair has built-in lumbar support. You can adjust the headrest and the lumbar support to suit you and your needs. The safety casters have a pressure-sensitive brake mechanism that keeps the chair in place when you stand up, and releases automatically when you sit down. The adjustable armrests of this swivel chair can help take the load off your shoulders and neck to reduce tension, strain and pain. The leather ages beautifully and acquires a nice patina over time. This chair has been tested for office use and meets the requirements for durability and stability set forth in the following standards: EN 1335 and ANSI/BIFMA x5.1. May be used with 

Tras obtener las descripciones vamos a separar en los 2 siguientes campos de código la columna `newData["product_title"]` en `title` y `short_descrip` respectivamente.

In [11]:
columnaTitulo = newData["product_title"]
titulos = []
for fila in columnaTitulo:
  division = fila.split(" ")
  titulos.append(division[0].strip())
titulos

['JÄRVFJÄLLET',
 'DRÖNA',
 'BESTIGA',
 'EKET',
 'SEVÄRD',
 'DRUVFLÄDER',
 'GULLIGAST',
 'BEKANT',
 'LURÖY',
 'KNYCK',
 'JUBLA',
 'FJÄLLHAVRE',
 'KOMPLEMENT',
 'LEDLJUS',
 'DRACAENA',
 'VALLENTUNA',
 'LINDÖJA',
 'DUGGREGN',
 'RINGBLOMMA',
 'HÄRLANDA',
 'IVRIG',
 'STORSLINGA',
 'LINDÖJA',
 'ENSEN',
 'BODBYN',
 'HITTARP',
 'SMYCKA',
 'SAKARIAS',
 'LEN',
 'MAXIMERA',
 'SMYCKA',
 'TOFTAN',
 'EKOLN',
 'NORRFLY',
 'SÖMNTUTA',
 'SINNLIG',
 'RYET',
 'HÖSTKVÄLL',
 'BERGMUND',
 'HÄGGEBY',
 'SILVERÅN',
 'BLÖTSNÖ',
 'SILVERHÖJDEN',
 'ÄDELHET',
 'HULTARP',
 'DINERA',
 'EKET',
 'SALVIKEN',
 'UPPLYST',
 'SKUBB',
 'HAUGA',
 'TRÅDFRI',
 'HITTARP',
 'KNOXHULT',
 'TROTTEN',
 'BENEDIKTA',
 'TRETUR',
 'HELMER',
 'FÖRBÄTTRA',
 'TRÅDFRI',
 'CHIAFRÖN',
 'LISTERBY',
 'HAVSTORP',
 'GLANSBLOMMA',
 'TOFTLUND',
 'REGISSÖR',
 'GRIMSLÖV',
 'SVARTISDAL',
 'SMYCKA',
 'LERHYTTAN',
 'SÖDERHAMN',
 'FINNALA',
 'FEJKA',
 'FJÄLLBRÄCKA',
 'VIDESPINNARE',
 'SÖMNTUTA',
 'NYSKÖLJD',
 'VOXTORP',
 'BRUNKRISSLA',
 'BERGMUND',
 'HÄR

In [12]:
descripcionCorta = [" ".join(i.split()[1:]) for i in newData["product_title"] ]
descripcionCorta

['Office chair with armrests',
 'Box - black 9 ¾x13 ¾x9 ¾ "',
 'Candlestick, set of 3 - black',
 'Suspension rail 13 3/4 "',
 'Tablecloth - dark blue 57x94 "',
 'Hanging planter - water hyacinth/gray 5 ½ "',
 'Baby gym',
 'Sit/stand underframe for table top - white 47 1/4x31 1/2 "',
 'Slatted bed base Full/Double',
 'Napkin holder - black',
 'Unscented candle - white 13 ¾ "',
 'Comforter, extra warm King',
 'Shoe shelf - white 19 5/8x13 3/4 "',
 'LED string light with 12 lights - outdoor/battery operated black',
 'Plant with pot - Lucky bamboo 2 "',
 'Storage seat section cover - Hillared beige',
 'Umbrella canopy - beige 118 1/8 "',
 'LED wall lamp - forest dark turquoise/light turquoise',
 'Roman blind - white/green/stripe 48x63 "',
 'Cover for armrest - Ljungen light green',
 'Glass - clear glass 15 oz',
 'LED chandelier, 8-armed - black/white',
 'Umbrella canopy - black 118 1/8 "',
 'Bath faucet with strainer - chrome plated',
 'Door - gray 12x40 "',
 'Drawer front - off-white 18x1

Para tener una columna de algún tipo de clasificación vamos a obtener una sección a partir de la columna `newData["breadcrumbs"]`. Como todos los breadcrumbs empiezan por Product/< seccion general >/< seccion especializada >/(...)/Nombre producto hemos decidido quedarnos con la seccion especializada ya que está presente en todos los productos. Sin embargo, como hay algunos productos cuyo breadcrumb es la cadena vacía a esos le vamos a añadir una sección creada por nosotros con el nombre "**Others**".

In [13]:
columnaSection = newData["breadcrumbs"]
secciones = []
for fila in columnaSection:
  division = fila.split("/")
  if len(division) < 3:
    secciones.append("Others") #Ponerle una sección inventada por defecto
  else:
    secciones.append(division[2]) #[0] products [1] seccion general [2] seccion especializada
secciones

['Chairs',
 'Storage boxes & baskets',
 'Candle holders & candles',
 'Shelving units, bookcases & storage options',
 'Table linen',
 'Flower pots & stands',
 'Baby',
 'Tables & desks',
 'Bed slats',
 'Napkins & napkin holders',
 'Candle holders & candles',
 'Bedding',
 'Storage solution systems',
 'Decorative lighting',
 'Plants & flowers',
 'Sofas & sectionals',
 'Umbrellas & gazebos',
 'Lamps & light fixtures',
 'Window treatments',
 'Sofas & sectionals',
 'Drinkware',
 'Lamps & light fixtures',
 'Umbrellas & gazebos',
 'Bathroom faucets',
 'Kitchen systems',
 'Kitchen systems',
 'Plants & flowers',
 'Others',
 'Mattresses',
 'Kitchen drawer organization',
 'Plants & flowers',
 'Storage containers',
 'Storage containers',
 'Armoires & wardrobes',
 'Bedding',
 'Candle holders & candles',
 'LED Light bulbs',
 'Candle holders & candles',
 'Chairs',
 'Kitchen systems',
 'Mirrors',
 'Decorative lighting',
 'Wall décor',
 'Candle holders & candles',
 'Kitchen wall organization & storage',


Vamos a crear un nuevo campo que sea el `sku` pero sin los puntos, es decir, un número entero, para que sea el ID de los productos y poder identificarlos.

In [14]:
columnaID = [int(x.replace(".","")) for x in newData["sku"]]
columnaID

[60510640,
 90467065,
 60504677,
 340047,
 10364090,
 241815,
 90484277,
 80263219,
 30292785,
 80045448,
 40154401,
 60458350,
 60257257,
 357429,
 60506841,
 30487405,
 50396126,
 30440875,
 80491075,
 90455270,
 50258323,
 40488452,
 396138,
 20281382,
 30266055,
 40266390,
 60496564,
 10384333,
 40143304,
 40265673,
 40496517,
 80349505,
 40493910,
 60332290,
 416886,
 50339961,
 80416415,
 30497579,
 486247,
 10267169,
 10267999,
 40421145,
 80298290,
 10416046,
 90444432,
 423957,
 80428852,
 10313227,
 440787,
 40472979,
 60474208,
 70437244,
 10266377,
 20372259,
 474763,
 10517663,
 40380475,
 50494363,
 492583,
 30400407,
 60475793,
 50409044,
 40475317,
 50500712,
 20420241,
 30342078,
 30405179,
 30473578,
 402945,
 40461594,
 30328322,
 60456431,
 90493385,
 80458542,
 10488929,
 40415743,
 40451057,
 10321294,
 90375527,
 451945,
 60455295,
 40266738,
 80130168,
 50360146,
 90266043,
 50459883,
 10467804,
 60490939,
 20502128,
 462152,
 30343737,
 40507083,
 70266058,
 60

##Creación del nuevo dataset

Los procesos anteriores nos dan como resultado unas listas que son las nuevas columnas de nuestro dataset modificado para ser usado en el sistema de recomendación y creamos un nuevo dataset que llamamos `datos`.


In [15]:
data = {
    'id' : columnaID,
    'sku' : newData["sku"],
    'name': titulos,
    'short_description' : descripcionCorta,
    'description' : descripciones,
    'price': newData["product_price"],
    'section': secciones,
    'url' : newData["product_url"]
}

datos = pd.DataFrame(data)
datos["image"] = ""
datos

,id,sku,name,short_description,description,price,section,url,image
0,60510640,605.106.40,JÄRVFJÄLLET,Office chair with armrests,10-year Warranty. Read about the terms in the ...,279.00,Chairs,https://www.ikea.com/us/en/p/jaervfjaellet-off...,
1,90467065,904.670.65,DRÖNA,"Box - black 9 ¾x13 ¾x9 ¾ """,A good size for various media and clothing acc...,2.99,Storage boxes & baskets,https://www.ikea.com/us/en/p/droena-box-black-...,
2,60504677,605.046.77,BESTIGA,"Candlestick, set of 3 - black","Nice as a table decoration, for example. Inclu...",24.99,Candle holders & candles,https://www.ikea.com/us/en/p/bestiga-candlesti...,
3,340047,003.400.47,EKET,"Suspension rail 13 3/4 """,The suspension rail works as a reinforcement a...,5.00,"Shelving units, bookcases & storage options",https://www.ikea.com/us/en/p/eket-suspension-r...,
4,10364090,103.640.90,SEVÄRD,"Tablecloth - dark blue 57x94 """,The tablecloth both protects the table and cre...,19.99,Table linen,https://www.ikea.com/us/en/p/sevaerd-tableclot...,
...,...,...,...,...,...,...,...,...,...
11814,90424146,904.241.46,HANDSKALAD,"Decoration, hand - natural",,12.99,Decorative accessories,https://www.ikea.com/us/en/p/handskalad-decora...,
11816,60460956,604.609.56,AURDAL,"Suspension rail - dark gray 25 5/8 """,Fits with AURDAL wardrobe system. The parts in...,5.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-suspension...,
11817,70462709,704.627.09,AURDAL,"Wire basket - white 22 1/4 """,Fits with AURDAL wardrobe system. The parts in...,10.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-wire-baske...,
11819,20435624,204.356.24,BODARP,"Drawer front - gray-green 15x15 """,25-year Limited Warranty. Read about the terms...,25.00,Kitchen systems,https://www.ikea.com/us/en/p/bodarp-drawer-fro...,


Para tener el dataset a mano por si hay algún problema lo guardamos en formato csv.

In [16]:
datos.to_csv("ikea_data_file.csv", index=False)
datos = pd.read_csv("ikea_data_file.csv", dtype={'id': int}) # Para que no haya ningún problema ponemos el id como int
datos.fillna("", inplace=True)
datos

,id,sku,name,short_description,description,price,section,url,image
0,60510640,605.106.40,JÄRVFJÄLLET,Office chair with armrests,10-year Warranty. Read about the terms in the ...,279.00,Chairs,https://www.ikea.com/us/en/p/jaervfjaellet-off...,
1,90467065,904.670.65,DRÖNA,"Box - black 9 ¾x13 ¾x9 ¾ """,A good size for various media and clothing acc...,2.99,Storage boxes & baskets,https://www.ikea.com/us/en/p/droena-box-black-...,
2,60504677,605.046.77,BESTIGA,"Candlestick, set of 3 - black","Nice as a table decoration, for example. Inclu...",24.99,Candle holders & candles,https://www.ikea.com/us/en/p/bestiga-candlesti...,
3,340047,003.400.47,EKET,"Suspension rail 13 3/4 """,The suspension rail works as a reinforcement a...,5.00,"Shelving units, bookcases & storage options",https://www.ikea.com/us/en/p/eket-suspension-r...,
4,10364090,103.640.90,SEVÄRD,"Tablecloth - dark blue 57x94 """,The tablecloth both protects the table and cre...,19.99,Table linen,https://www.ikea.com/us/en/p/sevaerd-tableclot...,
...,...,...,...,...,...,...,...,...,...
4488,90424146,904.241.46,HANDSKALAD,"Decoration, hand - natural",,12.99,Decorative accessories,https://www.ikea.com/us/en/p/handskalad-decora...,
4489,60460956,604.609.56,AURDAL,"Suspension rail - dark gray 25 5/8 """,Fits with AURDAL wardrobe system. The parts in...,5.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-suspension...,
4490,70462709,704.627.09,AURDAL,"Wire basket - white 22 1/4 """,Fits with AURDAL wardrobe system. The parts in...,10.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-wire-baske...,
4491,20435624,204.356.24,BODARP,"Drawer front - gray-green 15x15 """,25-year Limited Warranty. Read about the terms...,25.00,Kitchen systems,https://www.ikea.com/us/en/p/bodarp-drawer-fro...,


Podemos observar que el campo `description` es de tipo object y para poder procesar el texto tenemos que pasarlo a tipo *string*, que es lo que hacemos en los 2 siguientes campos.






In [17]:
datos.dtypes

id                     int64
sku                   object
name                  object
short_description     object
description           object
price                float64
section               object
url                   object
image                 object
dtype: object

In [18]:
datos['description'] = datos['description'].astype('string')
datos.dtypes

id                     int64
sku                   object
name                  object
short_description     object
description           string
price                float64
section               object
url                   object
image                 object
dtype: object

##Tokenización, BoW y Vectores de frecuencias

Una vez el campo de la descripción esta como string, procesamos la tabla creada anteriormente, llamada `datos`. Este proceso va a crear una nueva columna en la tabla llamada `processed_text`.



In [19]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

for row in datos.itertuples():

    text = word_tokenize(row[5]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = datos
preprocessedData['processed_text'] = preprocessedText

preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,sku,name,short_description,description,price,section,url,image,processed_text
0,60510640,605.106.40,JÄRVFJÄLLET,Office chair with armrests,10-year Warranty. Read about the terms in the ...,279.00,Chairs,https://www.ikea.com/us/en/p/jaervfjaellet-off...,,warranti read term warranti brochur the adjust...
1,90467065,904.670.65,DRÖNA,"Box - black 9 ¾x13 ¾x9 ¾ """,A good size for various media and clothing acc...,2.99,Storage boxes & baskets,https://www.ikea.com/us/en/p/droena-box-black-...,,a good size variou media cloth accessori ideal...
2,60504677,605.046.77,BESTIGA,"Candlestick, set of 3 - black","Nice as a table decoration, for example. Inclu...",24.99,Candle holders & candles,https://www.ikea.com/us/en/p/bestiga-candlesti...,,nice tabl decor exampl includ 1 candlestick ø3...
3,340047,003.400.47,EKET,"Suspension rail 13 3/4 """,The suspension rail works as a reinforcement a...,5.00,"Shelving units, bookcases & storage options",https://www.ikea.com/us/en/p/eket-suspension-r...,,the suspens rail work reinforc make easier saf...
4,10364090,103.640.90,SEVÄRD,"Tablecloth - dark blue 57x94 """,The tablecloth both protects the table and cre...,19.99,Table linen,https://www.ikea.com/us/en/p/sevaerd-tableclot...,,the tablecloth protect tabl creat decor tabl s...
...,...,...,...,...,...,...,...,...,...,...
4488,90424146,904.241.46,HANDSKALAD,"Decoration, hand - natural",,12.99,Decorative accessories,https://www.ikea.com/us/en/p/handskalad-decora...,,
4489,60460956,604.609.56,AURDAL,"Suspension rail - dark gray 25 5/8 """,Fits with AURDAL wardrobe system. The parts in...,5.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-suspension...,,fit aurdal wardrob system the part aurdal seri...
4490,70462709,704.627.09,AURDAL,"Wire basket - white 22 1/4 """,Fits with AURDAL wardrobe system. The parts in...,10.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-wire-baske...,,fit aurdal wardrob system the part aurdal seri...
4491,20435624,204.356.24,BODARP,"Drawer front - gray-green 15x15 """,25-year Limited Warranty. Read about the terms...,25.00,Kitchen systems,https://www.ikea.com/us/en/p/bodarp-drawer-fro...,,limit warranti read term limit warranti brochu...


Una vez se haya procesado todas las descripciones vamos a comparar la descripción con el nuevo texto procesado del segundo producto, por ejemplo.

In [20]:
preprocessedData.iloc[1]['description']

'A good size for various media and clothing accessories. Ideal in BILLY bookcase (depth 11") and BESTÅ storage (depth 15¾"). The box is easy to pull out regardless of where you place it since it has handles on all sides.'

In [21]:
preprocessedData.iloc[1]['processed_text']

'a good size variou media cloth accessori ideal billi bookcas depth 11 bestå storag depth 15¾ the box easi pull regardless place sinc handl side'

Podemos observar que se eliminan palabras y signos de puntuación que no aportan demasiado significado y otras palabras se simplifican, por ejemplo, pasar palabras del plural al singular.

Con el último dataset creado, llamado `preprocessedData` vamos a crear la bolsa de palabras (BoW) con el código a continuación.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW = bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

Finished


En los siguientes 4 campos de código veremos el tamaño, el contenido las palabras de la bolsa y como obtener una palabra de la bolsa a partir de un índice, respectivamente.

In [23]:
textsBoW.shape

(4493, 4305)

In [24]:
print(textsBoW)

  (0, 4272)	0.09656005261586706
  (0, 4269)	0.04163702640795095
  (0, 4155)	0.09344658902151312
  (0, 4041)	0.059538012313751755
  (0, 3832)	0.05896521494149075
  (0, 3830)	0.11989314635256203
  (0, 3803)	0.04488422045944059
  (0, 3788)	0.1418635387027889
  (0, 3782)	0.05748335128206566
  (0, 3777)	0.04661534809590779
  (0, 3773)	0.12806009029285417
  (0, 3736)	0.0779119527344872
  (0, 3712)	0.11546423701798667
  (0, 3690)	0.21246671291838165
  (0, 3667)	0.07348304339991184
  (0, 3615)	0.1426218922220224
  (0, 3557)	0.06381102007945283
  (0, 3555)	0.08145093458092666
  (0, 3545)	0.15791048937751917
  (0, 3385)	0.2492191883960797
  (0, 3347)	0.1161347080779348
  (0, 3301)	0.07007286847177946
  (0, 3267)	0.07800447816763348
  (0, 3210)	0.06771389131979397
  (0, 3120)	0.04563621162034741
  :	:
  (4492, 1263)	0.11126374509524269
  (4492, 1205)	0.06032531189114354
  (4492, 1193)	0.08135529594525091
  (4492, 1137)	0.11863189049913882
  (4492, 1098)	0.11210392097428908
  (4492, 1090)	0.088045

In [25]:
bagOfWordsModel.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['01',
 '10',
 '100',
 '1000',
 '1000mah',
 '1000w',
 '10050392',
 '10050454',
 '102',
 '1022',
 '104',
 '108',
 '10x10',
 '10x4',
 '10¼',
 '10⅝',
 '11',
 '110',
 '118',
 '11x11x5⅛',
 '11x4x2¾',
 '11x5½x5⅛',
 '11¾',
 '11¾x15¾',
 '11¾x17¾',
 '11⅞',
 '12',
 '120',
 '120v',
 '1219',
 '124',
 '125',
 '12520',
 '12x10⅝x4',
 '12x12',
 '12x16',
 '12¼x16⅛',
 '12¼x8¼x2⅜',
 '12⅝',
 '12⅝x14⅛',
 '13',
 '132',
 '1335',
 '13w',
 '13x13',
 '13x15',
 '13x21',
 '13¾',
 '13⅜',
 '13⅜x6¼',
 '14',
 '14073',
 '146',
 '14¼',
 '14½x11⅜',
 '14½x21¼x8¼',
 '14⅛',
 '14⅝',
 '15',
 '150',
 '152',
 '15372',
 '154',
 '15a',
 '15x15',
 '15¾',
 '15¾x15¾',
 '15¾x19⅝',
 '15¾x22⅞',
 '15¾x25⅝',
 '15¾x79½',
 '15⅜x25⅝',
 '16',
 '160',
 '16139',
 '164',
 '166',
 '16x20',
 '16x26',
 '16½',
 '16⅛x20⅛',
 '16⅞x15½',
 '17',
 '1728',
 '176',
 '17w',
 '17¼x13½x4¼',
 '17¾',
 '17⅜x17⅜',
 '17⅜x78',
 '18',
 '180',
 '1800',
 '186',
 '18½',
 '19',
 '190',
 '1900',
 '19w',
 '19¼',
 '19⅝',
 '19⅝x',
 '19⅝x13¾',
 '19⅝x19⅝',
 '19⅝x22⅞',
 '19⅝x

In [26]:
bagOfWordsModel.get_feature_names()[4000]

'unexpect'

Para realizar la recomendación necesitaremos ver cual es la distancia entre el producto del cual queremos recomendados y todos los demás ya que los productos cuya distancia sea la menor serán los que más se parecen y por ello recomendarlos. Para ello crearemos una matriz que será la que utilizaremos para calcular las distancias.

In [27]:
from sklearn.metrics import pairwise_distances

distance_matrix = pairwise_distances(textsBoW, textsBoW, metric='cosine')

In [28]:
print(distance_matrix.shape)
print(type(distance_matrix))

(4493, 4493)
<class 'numpy.ndarray'>


##Ejemplo 1: Busqueda de los productos más similares a otro del dataset

Con todo creado vamos a realizar nuestra primera búsqueda buscando productos parecidos a **DRÖNA** cuya descripción tenemos poco más arriba.

In [29]:
searchTitle = "DRÖNA"
indexOfTitle = preprocessedData[preprocessedData['name']==searchTitle].index.values[0]
indexOfTitle

1

Una vez obtenido el index del título a buscar vamos a calcular las distancias de todos los productos con respecto a ese producto.

In [30]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
distance_scores

[(0, 0.954356040396051),
 (1, 0.0),
 (2, 1.0),
 (3, 0.9958249286214872),
 (4, 0.9729334385140508),
 (5, 1.0),
 (6, 0.9747680266204836),
 (7, 1.0),
 (8, 0.9843019178775353),
 (9, 0.9639267649802037),
 (10, 0.9975029677676394),
 (11, 0.9750640838318542),
 (12, 0.9686643566571157),
 (13, 0.9881058876786845),
 (14, 0.9904043445786653),
 (15, 0.968493614983088),
 (16, 0.9807762040382733),
 (17, 0.990034280568656),
 (18, 0.991069955038723),
 (19, 1.0),
 (20, 0.9944378738478794),
 (21, 1.0),
 (22, 0.9807762040382733),
 (23, 0.9844316197320836),
 (24, 0.9750030653256795),
 (25, 0.969722033358802),
 (26, 0.9929307252400462),
 (27, 1.0),
 (28, 0.9623420101438802),
 (29, 0.9691347756442598),
 (30, 0.9965617678772818),
 (31, 0.9796305696623367),
 (32, 0.9702532763879987),
 (33, 0.9956045755338588),
 (34, 0.982615861651908),
 (35, 0.997468904610426),
 (36, 0.9948091171793463),
 (37, 1.0),
 (38, 0.9738455067361923),
 (39, 0.9567999467773347),
 (40, 0.9967236321439691),
 (41, 1.0),
 (42, 0.9870810261

Cuanto menor sea la distancia más parecidas son las descripciones por lo que vamos a ordenarlas de menor a mayor distancia.

In [31]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
ordered_scores

[(1, 0.0),
 (4341, 0.0),
 (1354, 0.43923026197843673),
 (707, 0.4765486778023229),
 (1358, 0.4765486778023229),
 (3507, 0.5547078362488037),
 (2175, 0.5847984784443933),
 (541, 0.5946097407184627),
 (3618, 0.6467711450778775),
 (3853, 0.6467711450778775),
 (944, 0.655358822394779),
 (452, 0.672268712097759),
 (4410, 0.6953691558232561),
 (2495, 0.7045406675991394),
 (2936, 0.7091440617590645),
 (657, 0.7094891753061623),
 (2109, 0.7198890588229421),
 (173, 0.7279085140000326),
 (1501, 0.7391270781703745),
 (2011, 0.7391270781703745),
 (3921, 0.7423064829118623),
 (1124, 0.7473403377936082),
 (3267, 0.7521368511090438),
 (1964, 0.7583883903351881),
 (2275, 0.7664248719354784),
 (2268, 0.7736742986553067),
 (2562, 0.7926442486377159),
 (879, 0.800107584978253),
 (2484, 0.8068455720196835),
 (519, 0.8081325381312877),
 (805, 0.8081325381312877),
 (2603, 0.8081325381312877),
 (420, 0.8082857241803585),
 (239, 0.8175313845140624),
 (2658, 0.8175313845140624),
 (3749, 0.8175313845140624),
 (

Vamos a quedarnos con las 10 primeras.

In [32]:
top_scores = ordered_scores[1:11]
top_scores

[(4341, 0.0),
 (1354, 0.43923026197843673),
 (707, 0.4765486778023229),
 (1358, 0.4765486778023229),
 (3507, 0.5547078362488037),
 (2175, 0.5847984784443933),
 (541, 0.5946097407184627),
 (3618, 0.6467711450778775),
 (3853, 0.6467711450778775),
 (944, 0.655358822394779)]

Obtenemos los índices de los productos anteriores.

In [33]:
top_indexes = [i[0] for i in top_scores]
top_indexes

[4341, 1354, 707, 1358, 3507, 2175, 541, 3618, 3853, 944]

Con los IDs anteriores vamos a obtener los nombres y la descripción corta de los productos parecidos.

In [34]:
preprocessedData['name'].iloc[top_indexes] + " " + preprocessedData['short_description'].iloc[top_indexes]

4341                     DRÖNA Box - white 9 ¾x13 ¾x9 ¾ "
1354    KUGGIS Storage box with lid - light green 10 ¼...
707     TJENA Storage box with lid - black 9 ¾x13 ¾x7 ¾ "
1358    TJENA Storage box with lid - white 9 ¾x13 ¾x7 ¾ "
3507    INSVEP Basket - plastic rattan brown 13x9 ¾x9 ½ "
2175    TÄTING Basket - banana leaves/natural 13 ¾x9 ¾...
541                    BILLY Corner hardware - galvanized
3618                         DRÖNA Box - white 13x15x13 "
3853                     DRÖNA Box - dark gray 13x15x13 "
944                    DRÖNA Box - black/white 13x15x13 "
dtype: object

Y ya tendríamos los productos parecidos al introducido basado en la descripción.

##Ejemplo 2: Busqueda de los productos más similares a partir de una descripción introducida por el usuario

Pero a nosotros nos gustaría poder meter una descripción y que te muestre productos que se acercan a esa descripción. Y es lo que se consigue hacer en el siguiente bloque de código. La solución es un poco fuerza bruta ya que no encontramos como hacerlo con un contenido exterior por lo que nuestro código introduce un objeto auxiliar al dataset con la descripción a buscar, una vez añadida se crea la columna de la descripción procesada, se crea la bolsa de palabras, la matriz de distancias y luego se busca ese.

In [35]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

import nltk
nltk.download('punkt')
nltk.download('stopwords')
SEARCHTITLE = "busqueda"
ps = PorterStemmer()

searchDescription = "a comfortable afordable chair"
nueva_row = {'id': 0, 'name':SEARCHTITLE, 'price':0, 'description':searchDescription}
datos = datos.append(nueva_row, ignore_index=True)
print("Añadido contenido a buscar")

newPreprocessedText = []
print("Procesando descripciones ...")
for row in datos.itertuples():

    text = word_tokenize(row[5]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    newPreprocessedText.append(text)

newPreprocessedData = datos
newPreprocessedData['processed_text'] = newPreprocessedText
print("Creada Tabla con texto procesado")

print("Creando Bag of Words ...")
bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(newPreprocessedData['processed_text'])
textsBoW = bagOfWordsModel.transform(newPreprocessedData['processed_text'])
print("Bag of Words creada")

print("Creando matriz de distancias ...")
distance_matrix = pairwise_distances(textsBoW, textsBoW, metric='cosine')
print("Creada Matriz para calcular las distancias")

indexOfTitle = newPreprocessedData[newPreprocessedData['name']==SEARCHTITLE].index.values[0]
print(indexOfTitle)
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
top_scores = ordered_scores[1:11]
top_indexes = [i[0] for i in top_scores]
print()

print("A CONTINUACIÓN TIENE LOS PRODUCTOS MÁS APROXIMADOS A SU BÚSQUEDA:\n")
newPreprocessedData['name'].iloc[top_indexes] + " " + newPreprocessedData['short_description'].iloc[top_indexes]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Añadido contenido a buscar
Procesando descripciones ...
Creada Tabla con texto procesado
Creando Bag of Words ...
Bag of Words creada
Creando matriz de distancias ...
Creada Matriz para calcular las distancias
4493

A CONTINUACIÓN TIENE LOS PRODUCTOS MÁS APROXIMADOS A SU BÚSQUEDA:



1068         ÖRFJÄLL Seat shell - white/Vissle light gray
2685                         OMTÄNKSAM Chair - light gray
1098    ÖRFJÄLL Child's desk chair - white/Vissle blue...
1774                            YNGVAR Chair - anthracite
4210      ÄPPLARÖ Chair, outdoor - foldable brown stained
3901            ÄPPLARÖ Armchair, outdoor - brown stained
654              SUNNEA Chair pad - Lofallet beige 14x1 "
1215                       SUNNEA Chair pad - gray 14x1 "
2661                 HILLARED Chair pad - beige 14x14x1 "
1617            BONDHOLMEN Chair, outdoor - foldable/gray
dtype: object

Una vez devuelto las recomendaciones borramos la fila, ya que es una fila "falsa" introducida a lo bruto. Y comprobamos que se ha eliminado.

In [36]:
newPreprocessedData.drop(newPreprocessedData.tail(1).index, inplace = True)
newPreprocessedData

,id,sku,name,short_description,description,price,section,url,image,processed_text
0,60510640,605.106.40,JÄRVFJÄLLET,Office chair with armrests,10-year Warranty. Read about the terms in the ...,279.00,Chairs,https://www.ikea.com/us/en/p/jaervfjaellet-off...,,warranti read term warranti brochur the adjust...
1,90467065,904.670.65,DRÖNA,"Box - black 9 ¾x13 ¾x9 ¾ """,A good size for various media and clothing acc...,2.99,Storage boxes & baskets,https://www.ikea.com/us/en/p/droena-box-black-...,,a good size variou media cloth accessori ideal...
2,60504677,605.046.77,BESTIGA,"Candlestick, set of 3 - black","Nice as a table decoration, for example. Inclu...",24.99,Candle holders & candles,https://www.ikea.com/us/en/p/bestiga-candlesti...,,nice tabl decor exampl includ 1 candlestick ø3...
3,340047,003.400.47,EKET,"Suspension rail 13 3/4 """,The suspension rail works as a reinforcement a...,5.00,"Shelving units, bookcases & storage options",https://www.ikea.com/us/en/p/eket-suspension-r...,,the suspens rail work reinforc make easier saf...
4,10364090,103.640.90,SEVÄRD,"Tablecloth - dark blue 57x94 """,The tablecloth both protects the table and cre...,19.99,Table linen,https://www.ikea.com/us/en/p/sevaerd-tableclot...,,the tablecloth protect tabl creat decor tabl s...
...,...,...,...,...,...,...,...,...,...,...
4488,90424146,904.241.46,HANDSKALAD,"Decoration, hand - natural",,12.99,Decorative accessories,https://www.ikea.com/us/en/p/handskalad-decora...,,
4489,60460956,604.609.56,AURDAL,"Suspension rail - dark gray 25 5/8 """,Fits with AURDAL wardrobe system. The parts in...,5.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-suspension...,,fit aurdal wardrob system the part aurdal seri...
4490,70462709,704.627.09,AURDAL,"Wire basket - white 22 1/4 """,Fits with AURDAL wardrobe system. The parts in...,10.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-wire-baske...,,fit aurdal wardrob system the part aurdal seri...
4491,20435624,204.356.24,BODARP,"Drawer front - gray-green 15x15 """,25-year Limited Warranty. Read about the terms...,25.00,Kitchen systems,https://www.ikea.com/us/en/p/bodarp-drawer-fro...,,limit warranti read term limit warranti brochu...


# Código para cargar a la API
Para crear una API con FastAPI extraemos el código a un fichero .py con el siguiente contenido:
1.   Método de inicialización `cargarDatos()`
2.   Método de recomendación `searchSimilarProducts(descrip, data)`
  *   `descrip` es la descripción a buscar productos similares
  *   `data` es el dataset cargado

In [37]:
import pandas as pd
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

SEARCHTITLE = "default"


def cargarDatos():
    """
    Carga el dataset en memoria. (SOLO se debería llamar 1 vez)

    :return: dataset de productos de IKEA
    """
    nltk.download('punkt')
    nltk.download('stopwords')
    dataset = pd.read_csv("ikea_data_file.csv", dtype={'id': int})
    dataset.fillna("", inplace=True)
    return dataset


def searchSimilarProducts(descrip, data):
    """
    Recibe una descripción y el dataset de productos a buscar

    :param descrip: Descripción a buscar items parecidos
    :param data: Dateset cargado de los productos de IKEA
    :return: listado de productos codificados en formato JSON
    """
    nueva_row = {'id': 0, 'name': SEARCHTITLE, 'price': 0, 'description': descrip}
    data = data.append(nueva_row, ignore_index=True)

    ps = PorterStemmer()
    newPreprocessedText = []
    for row in data.itertuples():
        text = word_tokenize(row[5])  # indice de la columna que contiene el texto
        # Remove stop words
        stops = set(stopwords.words("english"))
        text = [ps.stem(w) for w in text if w not in stops and w.isalnum()]
        text = " ".join(text)

        newPreprocessedText.append(text)

    newPreprocessedData = data
    newPreprocessedData['processed_text'] = newPreprocessedText

    bagOfWordsModel = TfidfVectorizer()
    bagOfWordsModel.fit(newPreprocessedData['processed_text'])
    textsBoW = bagOfWordsModel.transform(newPreprocessedData['processed_text'])

    distance_matrix = pairwise_distances(textsBoW, textsBoW, metric='cosine')

    indexOfTitle = newPreprocessedData[newPreprocessedData['name'] == SEARCHTITLE].index.values[0]
    distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
    ordered_scores = sorted(distance_scores, key=lambda x: x[1])
    top_scores = ordered_scores[1:11]
    top_indexes = [i[0] for i in top_scores]
    newPreprocessedData.drop(newPreprocessedData.tail(1).index, inplace=True)

    return json.dumps(datos.iloc[top_indexes].to_dict("records"))

Este sería un ejemplo de uso, que en vez de obtener la descripción del usuario se introduce como constante.

In [38]:
datos = cargarDatos()  # Esto carga el dataset y descarga algunas cosas
listadoJSON = searchSimilarProducts("a comfortable affordable chair", datos)  # esta sería como se invoca la función
print(listadoJSON)

listado = json.loads(listadoJSON)  # se lee el JSON de vuelta a una lista de diccionarios Python
print(listado)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[{"id": 90500970, "sku": "905.009.70", "name": "\u00d6RFJ\u00c4LL", "short_description": "Seat shell - white/Vissle light gray", "description": "The safety casters have a pressure-sensitive brake mechanism that keeps the chair in place when you stand up, and releases automatically when you sit down. You sit comfortably since the chair is adjustable in height. High-quality density foam will keep the chair comfortable for many years to come. To be completed with \u00d6RFJ\u00c4LL chair frame. Developed for domestic use.", "price": 29.99, "section": "Others", "url": "https://www.ikea.com/us/en/p/oerfjaell-seat-shell-white-vissle-light-gray-90500970/", "image": ""}, {"id": 50474341, "sku": "504.743.41", "name": "OMT\u00c4NKSAM", "short_description": "Chair - light gray", "description": "The chair is easy to get in and out of since the high wide back allows you to use the support of your arms. You sit comfortably thanks to the chair's shaped back and seat. The chair frame is made of solid w

# WEB SCRAPING IMÁGENES
El dataset que usamos no tiene imágenes por lo que vamos a ir recorriendo la columna `url` de los productos y obtener la url de la imagen por cada producto recorrido. Como el dataset es antiguo hay algunas urls que ya no están activas por lo que vamos a guardar un texto `No disponible` que indicará aquellas imágenes que no podamos obtener.

In [39]:
from bs4 import *
import requests

url_imagenes = []

def obtenerImagen(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    if soup.find(id="pip-media-1") is not None:
      image = soup.find(id="pip-media-1").find("button").find("span").find("img")
      try:
        image_link = image["src"]
      except:
        image_link = "No disponible"
    else:
      image_link = "No disponible"
    
    url_imagenes.append(image_link)

print("Esto tarda bastante, ten paciencia...(aprox 40 min)")
for url in preprocessedData['url']:
  obtenerImagen(url)
print("Listo!!")


Esto tarda bastante, ten paciencia...(aprox min)
Listo!!


Comprobamos que el tamaño de esta columna es igual al numero de filas de nuestro dataset, es decir, **4493**.

In [40]:
len(url_imagenes)

4493

Añadimos esos nuevos valores a la columna `image` que antes estaba vacía. Y comprobamos que se añadió correctamente.

In [41]:
preprocessedData["image"] = url_imagenes
preprocessedData

,id,sku,name,short_description,description,price,section,url,image,processed_text
0,60510640,605.106.40,JÄRVFJÄLLET,Office chair with armrests,10-year Warranty. Read about the terms in the ...,279.00,Chairs,https://www.ikea.com/us/en/p/jaervfjaellet-off...,https://www.ikea.com/us/en/images/products/jae...,warranti read term warranti brochur the adjust...
1,90467065,904.670.65,DRÖNA,"Box - black 9 ¾x13 ¾x9 ¾ """,A good size for various media and clothing acc...,2.99,Storage boxes & baskets,https://www.ikea.com/us/en/p/droena-box-black-...,https://www.ikea.com/us/en/images/products/dro...,a good size variou media cloth accessori ideal...
2,60504677,605.046.77,BESTIGA,"Candlestick, set of 3 - black","Nice as a table decoration, for example. Inclu...",24.99,Candle holders & candles,https://www.ikea.com/us/en/p/bestiga-candlesti...,https://www.ikea.com/us/en/images/products/bes...,nice tabl decor exampl includ 1 candlestick ø3...
3,340047,003.400.47,EKET,"Suspension rail 13 3/4 """,The suspension rail works as a reinforcement a...,5.00,"Shelving units, bookcases & storage options",https://www.ikea.com/us/en/p/eket-suspension-r...,https://www.ikea.com/us/en/images/products/eke...,the suspens rail work reinforc make easier saf...
4,10364090,103.640.90,SEVÄRD,"Tablecloth - dark blue 57x94 """,The tablecloth both protects the table and cre...,19.99,Table linen,https://www.ikea.com/us/en/p/sevaerd-tableclot...,No disponible,the tablecloth protect tabl creat decor tabl s...
...,...,...,...,...,...,...,...,...,...,...
4488,90424146,904.241.46,HANDSKALAD,"Decoration, hand - natural",,12.99,Decorative accessories,https://www.ikea.com/us/en/p/handskalad-decora...,https://www.ikea.com/us/en/images/products/han...,
4489,60460956,604.609.56,AURDAL,"Suspension rail - dark gray 25 5/8 """,Fits with AURDAL wardrobe system. The parts in...,5.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-suspension...,https://www.ikea.com/us/en/images/products/aur...,fit aurdal wardrob system the part aurdal seri...
4490,70462709,704.627.09,AURDAL,"Wire basket - white 22 1/4 """,Fits with AURDAL wardrobe system. The parts in...,10.00,Storage solution systems,https://www.ikea.com/us/en/p/aurdal-wire-baske...,https://www.ikea.com/us/en/images/products/aur...,fit aurdal wardrob system the part aurdal seri...
4491,20435624,204.356.24,BODARP,"Drawer front - gray-green 15x15 """,25-year Limited Warranty. Read about the terms...,25.00,Kitchen systems,https://www.ikea.com/us/en/p/bodarp-drawer-fro...,https://www.ikea.com/us/en/images/products/bod...,limit warranti read term limit warranti brochu...


Guardamos el nuevo dataset con las imagenes ya que tarda más de media hora obtener todos los enlaces de los productos.

In [42]:
preprocessedData.to_csv("ikea_data_file_imagenes.csv", index=False)